In [20]:
import json

# Open the dataset JSON
with open("dataSets/ingredient_and_instructions.json", "r") as dataSet:
    data = json.load(dataSet)

recipie = "" # The individual recipie
recipie_list = [] # Where we will store the recipie for training

for meal in data:
    # Get the recipie name and its instructions
    meal_info = data[meal]
    recipie += meal.replace("-", " ").title() + "\n\n"
    recipe_instructions = meal_info["instructions"]

    # Get the ingredients
    ingredients = meal_info["ingredient_sections"]

    for i in ingredients:
        # Retrieve ingredients and their details
        item = ""           # Dough
        qty = ""            # ½
        unit = ""           # cup
        ingredient = ""     # unsalted butter
        primary_unit = { "quantity": None, "display": None }

        # Some ingredients arent named, but are still listed as "1 cup" or similar
        # Additionally, if there is an ingredient name, it may already have a colon
        # in front of it, so we need to check for that before formatting
        if i["name"]:
            if i["name"][-1] != ":": # If there isnt a colon, add one
                item = i["name"] + ": \n"
            else: # Otherwise, just add the name
                item = i["name"] + " \n"
        else:
            item = ""

        recipie += item # Add the ingredient name to the recipie

        # Loop over each ingredient
        j = 0
        while j < len(i["ingredients"]):
            # i["ingredients"][0] may not exist, use try/except to prevent
            # any ListIndexErrors from accessing an index that doesn't exist
            try: primary_unit = i["ingredients"][j]["primary_unit"]
            except: primary_unit = { "quantity": None, "display": None } # Dummy object

            # Some recipies don't list an ingredient at all
            try: ingredient += i["ingredients"][j]["name"]
            except: ingredient += ""

            # If the try/except blocks above are successful, we can set the values
            if primary_unit["quantity"]: qty = primary_unit["quantity"] + " "
            if primary_unit["display"]: unit = primary_unit["display"] + " "

            # Construct the ingredient string
            recipie += qty + unit  + ingredient + "\n"
            ingredient = ""
            j += 1

        recipie += "\n"

    # Newline before adding instructions
    recipie += "\n"

    # Loop over the instructions and format them
    step = 1
    for text in recipe_instructions:
        instruction = text["display_text"]
        recipie += str(step) + ". " + instruction + "\n"
        step += 1

    # Add the recipie to the list
    recipie_list.append(recipie)

    # Free memory for the next recipie
    recipie = ""
    recipe_instructions = []
    ingredients = []
    item = ""
    qty = ""
    unit = ""
    ingredient = ""

# For testing purposes, we can output a random recipie
import random
print(recipie_list[random.randint(0, len(recipie_list) - 1)])

#print(recipie_list[0])


Loss: 1.720 — Avg: 1.726 — GPU Mem: 8571 MB:   4%|▎         | 2240/64000 [04:43<2:10:18,  7.90it/s]
Cheesesteak Skewers In A Box

½ white loaf bread
2 green bell peppers
3 red bell peppers
1 onion
1 oil
1 lb flank steak
1 lb salt
1 lb pepper
10 slices provolone cheese


1. Preheat oven to 350°F (180°C).
2. Turn the loaf on its end, and cut a square out, leaving a ½-inch (1 cm) thick edge. Remove the center of the bread.
3. Cut the hollowed bread piece into two and roll them down for compression.
4. Slice the bell peppers.
5. Cut the onions.
6. Thinly slice the flank steak.
7. Season the flank steak with salt and pepper to taste.
8. Heat oil in a pan over medium heat. Cook the flank steak until browned on all sides, about 2 minutes. Remove the steak from the pan.
9. Add the green peppers, red peppers, and onions, and cook until softened, about 4 minutes.
10. Season the peppers and onions with salt and pepper to taste.
11. Place the hollowed bread piece into the bread box.
12. Line the b

In [21]:
#CSV File 1 Record per row
#Record: 1 string with ingredients + recipe


# [[recipe1], [recipe2], [recipe3]]

import csv

with open("recipe.csv", 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    for recipie in recipie_list:
        csvwriter.writerow([recipie])


In [22]:
from aitextgen.TokenDataset import TokenDataset
from aitextgen.tokenizers import train_tokenizer
from aitextgen.utils import GPT2ConfigCPU
from aitextgen import aitextgen

file_name = "recipe.csv"

train_tokenizer(file_name)
tokenizer_file = "aitextgen.tokenizer.json"
config = GPT2ConfigCPU()
ai = aitextgen(tokenizer_file=tokenizer_file, config=config)
data = TokenDataset(file_name, tokenizer_file=tokenizer_file, block_size=64)
ai.train(data, batch_size=1000, num_steps=64000, generate_every=1000, save_every=5000)
ai.generate(prompt="steak")

You are tokenizing a CSV file, but you did not set line_by_line=True. Please change if unintended.
100%|██████████| 9922/9922 [00:00<00:00, 14438.56it/s]
pytorch_model.bin already exists in /trained_model and will be overwritten!
Windows does not support multi-GPU training. Setting to 1 GPU.
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


1,000 steps reached: generating sample texts.                                              
. Cut the salad from the salad pieces.                                                             
5. In a medium bowl, combine the balselly toast, red onion, onion, and garlic.
6. Bring the steaks to a boil.
7. Slowly scrape down
2,000 steps reached: generating sample texts.                                              
, garnish with fresh basil and serve.                                                              
6. Enjoy!
Healthicken As Mac Cheese

2 tablespoons unsalted butter
1 tablespoon sesame oil
2 tablespoons gradish mustard
2 teaspoons salt
2
3,000 steps reached: generating sample texts.                                              
 with the mustard and remaining 4 tablespoons of olive oil. Separate shoulders.                    
2. Scoop out the seeds and carrot to the bowl, then place your focacing over the sandwich.
3. Fry the
4,000 steps reached: generating sample texts.     

In [30]:
ai.generate(prompt="steak seasoning")

steak seasoning
1 lb salt and pepper
1 cup all-purpose flour
2 cup eggs
1 cup panko breadcrumbs
2 tablespoons olive oil
½ teaspoon ground turkey salt
½ teaspoon smoked paprika
